# Important: Read Me!

Before you start, create a new folder in the Files section called "data". Inside, put your three data .csv files (X_train, X_test, and y_train).

Then, put config.yaml and names.txt in the Files section as well.

# Imports

In [ ]:
!pip install numpy==1.26.0

In [ ]:
!pip install matplotlib==3.8.3
!pip install pandas==2.2.1
!pip install PyYAML==6.0.1
!pip install scikit_learn==1.2.2
!pip install torch==2.2.1
!pip install tqdm==4.66.2
!pip install xgboost==2.0.3

In [ ]:
import os
from yaml import safe_load
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

# Utils

In [ ]:
# Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# File utils
import os
from yaml import safe_load

# Torch
from torch import Tensor
from torch.utils.data import TensorDataset
from torch.nn.functional import normalize

cfg = safe_load(open("config.yaml", 'r'))

# Set seed
rng = np.random.default_rng(cfg["SEED"])


def read_csv_data(X_train_path, y_train_path, X_test_path, val):
    # Read in the dataset
    X_train = pd.read_csv(X_train_path)
    X_test = pd.read_csv(X_test_path)
    y_train = pd.read_csv(y_train_path)["label"]

    # Remove any rows from the train set that contain NA values
    X_train = X_train.dropna(axis=0)
    y_train = y_train.iloc[X_train.index]

    # (val)% of all data set aside for validation
    n_val = int(val * (len(X_train) + len(X_test)))
    val_idx = rng.choice(np.arange(len(X_train)), n_val, replace=False)

    return X_train, y_train, X_test, list(val_idx)


def train_val_split(X_train, y_train, val_idx):
    X_train = X_train.reset_index(drop=True)

    v = X_train.index.isin(val_idx)

    X_val = X_train[v]
    y_val = y_train[v]

    X_train = X_train[~v]
    y_train = y_train[~v]

    return X_train, y_train, X_val, y_val


def eliminate_variables(X, group_start, group_end, new_ftr_name):
    idx = list(X.columns).index(group_start)
    curr_var = group_start
    to_drop = []
    while curr_var != group_end:
        to_drop.append(curr_var)
        idx += 1
        curr_var = list(X.columns)[idx]
    if new_ftr_name:
        X[new_ftr_name] = X[to_drop].sum(axis=1)
    X = X.drop(columns=to_drop)
    return X


def zero_out_negatives(X, ftr_name):
    vals = X[ftr_name].unique()
    for v in vals:
        if v < 0:
            X[ftr_name] = X[ftr_name].replace(v, 0)
    return X


def feature_transform(X):
    # Rewrite fieldwork start and end dates as numbers
    fw_start = X["fw_start"].apply(lambda x: x // 100 + (x % 100) / 12)
    fw_end = X["fw_end"].apply(lambda x: x // 100 + (x % 100) / 12)

    # Replace field work start and end with duration
    fw_duration = fw_end - fw_start
    X = X.assign(fw_duration=fw_duration)
    X = X.drop(columns=["fw_start", "fw_end"])

    # Aggregate some integer-encoded categorical variables (not of type "object")
    X = eliminate_variables(X, "v22", "v31", "neighbor_discrim")
    X = eliminate_variables(X, "v31", "v38", "distrust")
    X = eliminate_variables(X, "v40", "v45a", "job_standard")
    X = eliminate_variables(X, "v46", "v51", "work_ethic")
    X = eliminate_variables(X, "v72", "v79", "patriarch_belief")
    X = eliminate_variables(X, "v83", "v96", "children_virtues")
    X = eliminate_variables(X, "v97", "v102", "poli_activism")
    X = eliminate_variables(X, "v115", "v133", "society_conf")
    X = eliminate_variables(X, "v133", "v144", "democracy")
    X = eliminate_variables(X, "v146", "v149", "poli_system")
    X = eliminate_variables(X, "v164", "v171", "community")

    # Subtract each of the "none mentioned" variables from their relevant groups
    X = zero_out_negatives(X, "v45a")
    X["job_standard"] = X["job_standard"] - X["v45a"]
    X = X.drop(columns=["v45a"])

    X = zero_out_negatives(X, "v96")
    X["children_virtues"] = X["children_virtues"] - X["v96"]
    X = X.drop(columns=["v96"])

    X = X.drop(columns=["v108", "v109", "v110", "v111"])

    # Add/subtract DK (don't know) values based on topic: subtract if higher rating means individual perceives a
    # high level of corruption in their country; add otherwise
    to_subtract = ["v176", "v180", "v181"]
    to_add = ["v177", "v178", "v179", "v182", "v183"]
    corruption = X[to_add].sum(axis=1) - X[to_subtract].sum(axis=1)

    # Compute corruption values for both the original and "don't know" responses
    to_subtract = [var + "_DK" for var in to_subtract]
    to_add = [var + "_DK" for var in to_add]
    corruption_dk = X[to_add].sum(axis=1) - X[to_subtract].sum(axis=1)

    # Replace these variables with the corruption feature
    X = X.assign(corruption=(corruption + corruption_dk))
    X = X.drop(columns=to_add)
    X = X.drop(columns=to_subtract)

    # Select variables up to 224_DK, along with the new variables we selected
    # Specify the new feature name as None to not make a new aggregate variable
    X = eliminate_variables(X, "v225", "fw_duration", None)

    return X


def drop(X):
    """
    Drop variables with zero variance and/or whose values are modified/flagged
    :param X: pd.DataFrame; original dataset.
    :return: pd.DataFrame, with columns dropped.
    """
    # Drop the id and country columns (redundant information)
    vars_to_drop = ["country"]
    for var in X.columns:
        vals = X[var].unique()

        # Remove variables with only one unique value
        if len(vals) == 1 or X[var].dtype != "O" and len(vals[vals >= 0]) == 1:
            vars_to_drop.append(var)

        # Drop modified and/or flagged variables
        elif "DE" in var:
            vars_to_drop.append(var)
        elif "f" in var and "fw" not in var:
            vars_to_drop.append(var)

    X = X.drop(columns=vars_to_drop)
    return X


def preprocess(X_train, y_train, X_test, val_idx, preproc_dir=None):
    # Replace -1's in y_train with 0's
    y_train = y_train.replace(-1, 0)

    # The drop() function removes variables with only one unique value; this is checked across all observations,
    # i.e. both train and test sets
    print("Removing constant, modified, and flagged variables...")
    X = pd.concat([X_train, X_test])
    X = drop(X)

    # Retrieve the train and test sets
    X_train = X.iloc[:len(X_train)]
    X_test = X.iloc[len(X_train):]

    # Do some feature engineering
    print("Applying feature engineering...")
    X_train = feature_transform(X_train)
    X_test = feature_transform(X_test)

    # Handle the categorical variables for XGBoost
    print("Ensuring categorical variable types...")
    X_train = categorical_dmatrix(X_train)
    X_test = categorical_dmatrix(X_test)

    # Train-val split
    print("Generating a train-val data split...")
    X_train, y_train, X_val, y_val = train_val_split(X_train, y_train, val_idx)

    # Write the dfs to .csv files if applicable
    if preproc_dir:
        if len(os.listdir("preprocessed")) == 0 and not os.path.exists(preproc_dir):
            os.makedirs(preproc_dir)

            X_train_path = os.path.join(preproc_dir, "X_train.csv")
            X_val_path = os.path.join(preproc_dir, "X_val.csv")
            X_test_path = os.path.join(preproc_dir, "X_test.csv")

            X_train.to_csv(X_train_path, index=False)
            X_val.to_csv(X_val_path, index=False)
            X_test.to_csv(X_test_path, index=False)

            print("Preprocessed datasets saved to")
            print("-- X_train: {}".format(X_train_path))
            print("-- X_val: {}".format(X_val_path))
            print("-- X_test: {}".format(X_test_path))

    return X_train, y_train, X_val, y_val, X_test


def categorical_dmatrix(X):
    for var in X.columns:
        if X[var].dtype == "O":
            X[var] = X[var].astype("category")
    return X


def select_features(X_train, X_val, X_test, xgb_model):
    """
    Apply a feature selection transformation (based on importance matrix from XGBoost) on train dnd test sets.
    :param X_train: pd.DataFrame; train set
    :param X_test: pd.DataFrame; test set
    :param xgb_model: XGBoost Booster model.
    :return: (pd.DataFrame, pd.DataFrame) (with only the selected features from XGBoost)
    """
    # Get feature importance from XGBoost model (descending order)
    imp = xgb_model.feature_importances_
    ftr_idx = np.where(imp > cfg["XGBOOST"]["SELECTION"]["IMP_THRESH"])
    ftr_select = []

    # Get features with non-zero importance
    print("Selecting features based on gain...")
    for idx in ftr_idx:
        ftr_select.append(xgb_model.feature_names_in_[idx])

    ftr_select = list(ftr_select[0])
    print("{} features selected from XGBoost.".format(len(ftr_select)))

    # Feature selection
    X_train = X_train[ftr_select]
    X_val = X_val[ftr_select]
    X_test = X_test[ftr_select]

    return X_train, X_val, X_test


def write_pred_csv(pred_df, dest):
    assert not os.path.exists(dest), "The prediction directory {} already exists.".format(dest)
    if not os.path.exists(dest):
        os.makedirs(dest)
        pred_path = os.path.join(dest, "pred.csv")
        pred_df.to_csv(pred_path, index_label="id")


def pandas2numpy(X_train, X_val, X_test):
    # One-hot-encode
    X = pd.concat([X_train, X_val, X_test])
    X = label_encode(X)

    n_train, n_val = len(X_train), len(X_val)

    # Scale features between 0 and 1
    mean = np.tile(np.mean(X, axis=1), (X.shape[1], 1)).transpose()
    std = np.tile(np.std(X, axis=1), (X.shape[1], 1)).transpose()
    X = X * mean / std

    X_train = X.iloc[:n_train]
    X_val = X.iloc[n_train:n_train+n_val]
    X_test = X.iloc[n_train+n_val:]

    return np.array(X_train), np.array(X_val), np.array(X_test)


def pandas2torch(X, y):
    X = label_encode(X)
    # Cast booleans in X to ints (for numpy conversion)
    for var in X.columns:
        if X[var].dtype == "bool":
            X[var] = X[var].astype(int)

    X_tensor = Tensor(X.to_numpy())
    y_tensor = Tensor(y.to_numpy())
    return TensorDataset(normalize(X_tensor), y_tensor)


def label_encode(X):
    le = LabelEncoder()
    for var in X.columns:
        if X[var].dtype in ["object", "category"]:
            X[var] = le.fit_transform(X[var])
    return X

# Models

In [ ]:
from yaml import safe_load

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Torch
import torch
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam, Rprop
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

cfg = safe_load(open("config.yaml", 'r'))

rng = np.random.default_rng(cfg["SEED"])


class XGBoost():
    def __init__(self):
        super().__init__()
        self.model = "xgboost"
        self.trained = None

    def train(self, X_train, y_train, X_val, y_val, xgb_cfg, cv=True, pre_test=None):
        X_pre_test, y_pre_test = None, None

        # If we want to set aside a preemptive test set for further evaluation purposes
        if pre_test > 0:
            n_train = len(X_train)
            pre_test_idx = rng.choice(np.arange(n_train), size=int(pre_test * n_train), replace=False)
            pre_test_data = X_train.index.isin(pre_test_idx)
            X_pre_test = X_train.iloc[pre_test_data]
            y_pre_test = y_train.iloc[pre_test_data]
            X_train_new = X_train.iloc[~pre_test_data]
            y_train_new = y_train.iloc[~pre_test_data]
        else:
            X_train_new = X_train
            y_train_new = y_train

        # Watchlist (monitor over-fitting)
        eval_set = [(X_train_new, y_train_new), (X_val, y_val)]

        # Define the model with parameters from the config file
        model = XGBClassifier(n_estimators=xgb_cfg["ROUNDS"], objective="multi:softprob", eval_metric="mlogloss",
                              enable_categorical=xgb_cfg["CATEGORICAL"], tree_method="hist",
                              colsample_bytree=xgb_cfg["SUBSET"], n_jobs=cfg["N_JOBS"], early_stopping_rounds=50)

        params = {"max_depth": xgb_cfg["DEPTH"], "eta": xgb_cfg["LR"], "min_child_weight": xgb_cfg["MIN_CHILD_WGT"]}

        # Grid search cross-validation
        if cv:
            xgb_search = GridSearchCV(model, params, n_jobs=cfg["N_JOBS"])
            xgb_search.fit(X_train_new, y_train_new, eval_set=eval_set, verbose=2)

            best_model = xgb_search.best_estimator_
            self.trained = best_model

            # Output best set of parameters
            for param in xgb_search.best_params_:
                print("{}: {}".format(param, xgb_search.best_params_[param]))

        # Single train-test pass
        else:
            assert len(xgb_cfg["DEPTH"]) == 1, \
                "Depth parameter has length {} when 1 is required.".format(len(xgb_cfg["DEPTH"]))
            assert len(xgb_cfg["LR"]) == 1, \
                "Learning rate parameter has length {} when 1 is required.".format(len(xgb_cfg["LR"]))
            assert len(xgb_cfg["MIN_CHILD_WGT"]) == 1, \
                "Min child wgt parameter has length {} when 1 is required.".format(len(xgb_cfg["MIN_CHILD_WGT"]))

            model.set_params(max_depth=xgb_cfg["DEPTH"][0], learning_rate=xgb_cfg["LR"][0],
                             min_child_weight=xgb_cfg["MIN_CHILD_WGT"][0])
            self.trained = model.fit(X_train_new, y_train_new, eval_set=eval_set)

        # Evaluate best model on preemptive test set
        val_pred = self.trained.predict_proba(X_val)
        val_pred_class = np.argmax(val_pred, axis=1)

        eval_metrics = {
            "logloss": log_loss(y_val, val_pred),
            "accuracy": accuracy_score(y_val, val_pred_class)
        }

        # Evaluation metrics on the preemptive test set, if applicable
        if pre_test > 0:
            pre_test_pred = self.trained.predict_proba(X_pre_test)
            pre_test_pred_class = np.argmax(pre_test_pred, axis=1)
            eval_metrics["pre_test_logloss"] = log_loss(y_pre_test, pre_test_pred)
            eval_metrics["pre_test_accuracy"] = accuracy_score(y_pre_test, pre_test_pred_class)

        return eval_metrics

    def predict(self, X_test):
        pred = self.trained.predict_proba(X_test)

        # Retrieve column names for prediction dataframe
        names_file = open("names.txt", "r")
        columns = names_file.readlines()
        names_file.close()

        # Remove the newline characters
        for i in range(len(columns) - 1):
            columns[i] = columns[i][:-1]

        # Construct prediction dataframe
        pred_df = pd.DataFrame(pred, columns=columns)

        return pred_df


class RandomForest:
    def __init__(self):
        self.trained = None

    def train(self, X_train, y_train, X_val, y_val):
        rf = cfg["RF"]

        self.trained = RandomForestClassifier(n_estimators=rf["N_TREES"], criterion="log_loss", max_depth=9,
                                              min_samples_leaf=rf["MIN_CHILD_WGT"], max_features=rf["MAX_FTRS"],
                                              n_jobs=cfg["N_JOBS"], verbose=1)
        self.trained.fit(X_train, y_train)
        train_pred = self.trained.predict_proba(X_train)
        train_pred_class = np.argmax(train_pred, axis=1)
        print("Random forest log loss (train):", log_loss(y_train, train_pred))
        print("Random forest accuracy (train):", accuracy_score(y_train, train_pred_class))

        val_pred = self.trained.predict_proba(X_val)
        pred_class = np.argmax(val_pred, axis=1)
        return {
            "logloss": log_loss(y_val, val_pred),
            "accuracy": accuracy_score(y_val, pred_class)
        }


class NN(torch.nn.Module):
    def __init__(self, dims):
        super(NN, self).__init__()

        self.b1 = torch.nn.BatchNorm1d(dims[0])
        self.b2 = torch.nn.BatchNorm1d(dims[1])
        self.b3 = torch.nn.BatchNorm1d(dims[2])
        self.b4 = torch.nn.BatchNorm1d(dims[3])
        self.b5 = torch.nn.BatchNorm1d(dims[4])
        self.b6 = torch.nn.BatchNorm1d(dims[5])

        self.fc1 = torch.nn.Linear(dims[0], dims[1])
        self.fc2 = torch.nn.Linear(dims[1], dims[2])
        self.fc3 = torch.nn.Linear(dims[2], dims[3])
        self.fc4 = torch.nn.Linear(dims[3], dims[4])
        self.fc5 = torch.nn.Linear(dims[4], dims[5])
        self.fc6 = torch.nn.Linear(dims[5], dims[6])

        # Initialize weights
        torch.nn.init.kaiming_normal_(self.fc1.weight)
        torch.nn.init.kaiming_normal_(self.fc2.weight)
        torch.nn.init.kaiming_normal_(self.fc3.weight)
        torch.nn.init.kaiming_normal_(self.fc4.weight)
        torch.nn.init.kaiming_normal_(self.fc5.weight)
        torch.nn.init.kaiming_normal_(self.fc6.weight)

    def forward(self, x):
        x = self.b1(x)
        x = torch.nn.Dropout(cfg["NN"]["DROPOUT"])(x)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.b2(x)
        x = torch.nn.Dropout(cfg["NN"]["DROPOUT"])(x)
        x = self.fc2(x)
        x = F.relu(x)

        x = self.b3(x)
        x = torch.nn.Dropout(cfg["NN"]["DROPOUT"])(x)
        x = self.fc3(x)
        x = F.relu(x)

        x = self.b4(x)
        x = torch.nn.Dropout(cfg["NN"]["DROPOUT"])(x)
        x = self.fc4(x)
        x = F.relu(x)

        x = self.b5(x)
        x = torch.nn.Dropout(cfg["NN"]["DROPOUT"])(x)
        x = self.fc5(x)
        x = F.relu(x)

        x = self.b6(x)
        x = torch.nn.Dropout(cfg["NN"]["DROPOUT"])(x)
        x = self.fc6(x)
        x = F.log_softmax(x, dim=1)

        return x


class NeuralNetwork:
    def __init__(self):
        self.net = None
        self.epochs = cfg["NN"]["EPOCHS"]
        self.model = "nn"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train_one_epoch(self, train_loader, optim, loss_fn):
        loss_sum = 0

        for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
            # Data
            x, y = data
            x = x.to(self.device)
            y = y.type(torch.LongTensor).to(self.device)
            optim.zero_grad()

            # Train
            pred = self.net(x)

            # Training loss with L2 regularization
            loss = loss_fn(pred, y)
            loss.backward()
            optim.step()
            loss_sum += loss.item()

        # Report the average training loss across the batches
        return loss_sum / len(train_loader)

    def train(self, X_train, y_train, X_val, y_val, return_history=False):
        torch.set_num_threads(6)

        hidden_dims = cfg["NN"]["H_DIM"]
        dims = [len(X_train.columns)] + hidden_dims + [y_train.nunique()]
        self.net = NN(dims).to(self.device)

        dtrain = pandas2torch(X_train, y_train)
        dval = pandas2torch(X_val, y_val)

        train_loader = DataLoader(dtrain, batch_size=cfg["NN"]["BATCH_SIZE"], shuffle=True)
        val_loader = DataLoader(dval, batch_size=cfg["NN"]["BATCH_SIZE"], shuffle=False)

        # optim = SGD(self.net.parameters(), lr=cfg["NN"]["LR"], weight_decay=cfg["NN"]["L2_REG"],
        #             momentum=cfg["NN"]["MOMENTUM"])
        optim = Rprop(self.net.parameters(), lr=cfg["NN"]["RPROP_LR"])
        # scheduler = CosineAnnealingLR(optimizer=optim, T_max=cfg["NN"]["EPOCHS"])

        loss_fn = torch.nn.NLLLoss().to(self.device)

        # TensorBoard writer
        writer = SummaryWriter()

        val_loss_sum = 0
        best_val_loss = 10000
        curr_acc = 0
        train_history, val_history = [], []
        for epoch in range(self.epochs):
            print("Epoch {}:".format(epoch + 1))

            self.net.train(True)
            # scheduler.step()
            train_loss = self.train_one_epoch(train_loader, optim, loss_fn)
            train_history.append(train_loss)
            writer.add_scalar("logloss_train", train_loss, epoch)
            print("Training loss: {}".format(train_loss))

            self.net.eval()
            correct = 0
            with torch.no_grad():
                for i, data in tqdm(enumerate(val_loader), total=len(val_loader)):
                    x, y = data
                    x = x.to(self.device)
                    y = y.type(torch.LongTensor).to(self.device)
                    pred = self.net(x)
                    loss = loss_fn(pred, y)
                    val_loss_sum += loss.item()

                    # Validation accuracy
                    pred_class = torch.argmax(pred, dim=1)
                    correct += (pred_class == y).float().sum()

            val_loss = val_loss_sum / len(val_loader)
            val_loss_sum = 0
            val_acc = correct / len(y_val)
            val_history.append(val_loss)
            writer.add_scalar("logloss_val", val_loss, epoch)
            print("Validation loss: {}".format(val_loss))
            print("Validation accuracy: {}".format(val_acc))

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                curr_acc = val_acc
                model_path = 'model.pt'.format(epoch + 1)
                torch.save(self.net.state_dict(), model_path)

        writer.flush()
        writer.close()

        eval_metrics = {
            "logloss": best_val_loss,
            "accuracy": curr_acc
        }

        if return_history:
            eval_metrics["loss_history"] = {
                "train": train_history,
                "val": val_history
            }

        return eval_metrics

    def predict(self, X_test):
        pass


# Driver Code

In [ ]:
import os
from yaml import safe_load
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

# Config
cfg = safe_load(open("config.yaml", "r"))


if __name__ == "__main__":
    # Read data
    X_train, y_train, X_test, val_idx = read_csv_data(cfg["X_TRAIN"], cfg["Y_TRAIN"], cfg["X_TEST"], cfg["VAL"])

    # Get datetime for saving preprocessed datasets
    if not os.path.exists("preprocessed"):
        os.makedirs("preprocessed")
    preproc_dir = os.path.join("preprocessed", datetime.now().strftime("%X").replace(":", "_"))
    X_train, y_train, X_val, y_val, X_test = preprocess(X_train, y_train, X_test, val_idx, preproc_dir)

    # XGBoost for feature selection
    xgb_select = XGBoost()
    print("Fitting XGBoost (selection). {} features".format(len(X_train.columns)))
    xgb_select_metrics = xgb_select.train(X_train, y_train, X_val, y_val, cfg["XGBOOST"]["SELECTION"], cv=False,
                                          pre_test=cfg["PRE_TEST"])
    print("XGBoost log loss:", xgb_select_metrics["logloss"])
    print("XGBoost accuracy:", xgb_select_metrics["accuracy"])

    # If we did a pre-test
    if cfg["PRE_TEST"] > 0:
        print("XGBoost log loss (pre-test):", xgb_select_metrics["pre_test_logloss"])
        print("XGBoost accuracy (pre-test):", xgb_select_metrics["pre_test_accuracy"])

    # Prediction
    print("Predicting with XGBoost...")
    if not os.path.exists("predictions"):
        os.makedirs("predictions")
    pred_df = xgb_select.predict(X_test)
    pred_dir = os.path.join("predictions", datetime.now().strftime("%X").replace(":", "_"))
    write_pred_csv(pred_df, pred_dir)

    # Select important features from XGBoost
    X_train, X_val, X_test = select_features(X_train, X_val, X_test, xgb_select.trained)

    # Random forest
    # rf = RandomForest()
    # print("Fitting random forest (prediction). {} features".format(len(X_train.columns)))
    # X_train_np, X_val_np, X_test_np = pandas2numpy(X_train, X_val, X_test)
    # rf_metrics = rf.train(X_train_np, y_train, X_val_np, y_val)
    # print("Random forest log loss:", rf_metrics["logloss"])
    # print("Random forest accuracy:", rf_metrics["accuracy"])

    # Neural network training
    neural_net = NeuralNetwork()
    print("Fitting MLP (prediction). {} features".format(len(X_train.columns)))
    dtrain = pandas2torch(X_train, y_train)
    dval = pandas2torch(X_val, y_val)
    nn_metrics = neural_net.train(X_train, y_train, X_val, y_val, return_history=True)
    print("MLP log loss:", nn_metrics["logloss"])
    print("MLP accuracy:", nn_metrics["accuracy"])

    # Plot neural net train/val loss curves
    t = np.arange(neural_net.epochs) + 1
    plt.plot(t, nn_metrics["loss_history"]["train"], label="train loss")
    plt.plot(t, nn_metrics["loss_history"]["val"], label="val loss")

    # Plot XGBoost loss for reference
    xgb_loss = np.repeat(xgb_select_metrics["logloss"], neural_net.epochs)
    plt.plot(t, xgb_loss, label="XGBoost loss", linestyle="dashed")

    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Log Loss")
    plt.title("Log Loss (Train/Val)")
    plt.savefig("nn_loss.png")


<IPython.core.display.Javascript object>

<ipython-input-5-81f1d15ad6d4>:23: DtypeWarning: Columns (105) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv(X_train_path)


Removing constant, modified, and flagged variables...
Applying feature engineering...
Ensuring categorical variable types...
Generating a train-val data split...
Fitting XGBoost (selection). 157 features
[0]	validation_0-mlogloss:1.57607	validation_1-mlogloss:1.57642
[1]	validation_0-mlogloss:1.54523	validation_1-mlogloss:1.54578
[2]	validation_0-mlogloss:1.51656	validation_1-mlogloss:1.51726
[3]	validation_0-mlogloss:1.48931	validation_1-mlogloss:1.49031
[4]	validation_0-mlogloss:1.46415	validation_1-mlogloss:1.46529
[5]	validation_0-mlogloss:1.44079	validation_1-mlogloss:1.44201
[6]	validation_0-mlogloss:1.41888	validation_1-mlogloss:1.42036
[7]	validation_0-mlogloss:1.39777	validation_1-mlogloss:1.39981
[8]	validation_0-mlogloss:1.37813	validation_1-mlogloss:1.38031
[9]	validation_0-mlogloss:1.35896	validation_1-mlogloss:1.36186
[10]	validation_0-mlogloss:1.34162	validation_1-mlogloss:1.34468
[11]	validation_0-mlogloss:1.32450	validation_1-mlogloss:1.32820
[12]	validation_0-mlogloss

<ipython-input-5-81f1d15ad6d4>:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[var] = le.fit_transform(X[var])
<ipython-input-5-81f1d15ad6d4>:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[var] = le.fit_transform(X[var])


Epoch 1:


...

Training loss: 1.1762536764144897


100%|██████████| 6/6 [00:00<00:00, 159.14it/s]


Validation loss: 1.1785940527915955
Validation accuracy: 0.4710090160369873
Epoch 389:


 10%|█         | 1/10 [00:00<00:00, 50.40it/s]


KeyboardInterrupt: 